In [1]:
import json
import os
from datetime import datetime

# 1. 파일 경로 정의
input_dir = '/home/ds4_sia_nolb/#FINAL_POLARIS/06_Geo_coding/re_combined_data_by_year'
output_dir = '/home/ds4_sia_nolb/#FINAL_POLARIS/06_Geo_coding/re_combined_data_by_year_mapping_v1_jsonl/'
geo_data_dir = '/home/ds4_sia_nolb/code_from_8_19/sh/geocoding/Geosjon_data'

# 2. GeoJSON 파일에서 좌표 데이터 불러와 변수에 저장
# 각 위치에 해당하는 정보와 타입을 함께 저장하고, 키(key_name)의 공백을 제거합니다.
def load_geojson_features(file_path, key_name, geo_type):
    """GeoJSON 파일에서 특정 키를 기준으로 맵을 생성하고, 지리적 단위를 추가하며, 공백을 제거합니다."""
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            geojson_data = json.load(f)
        feature_map = {
            # GeoJSON 데이터의 이름에서 공백 제거 후 맵의 키로 사용
            feature['properties'].get(key_name, '').replace(' ', ''): {
                "name": feature['properties'].get(key_name, ''),  # 원본 이름도 함께 저장
                "coordinates": feature['geometry']['coordinates'],
                "type": geo_type
            }
            for feature in geojson_data['features']
        }
        return feature_map
    except FileNotFoundError:
        print(f"오류: GeoJSON 파일이 존재하지 않습니다: {file_path}")
        return None
    except json.JSONDecodeError:
        print(f"오류: GeoJSON 파일 '{file_path}'이(가) 유효한 JSON 형식이 아닙니다.")
        return None

try:
    # 단위별로 맵 생성
    si_do_map = load_geojson_features(os.path.join(geo_data_dir, 'dprk_si_do.geojson'), 'NL_NAME_1', 'si_do')
    si_gun_map = load_geojson_features(os.path.join(geo_data_dir, 'dprk_si_gun.geojson'), 'NL_NAME_2', 'si_gun')
    points_map = load_geojson_features(os.path.join(geo_data_dir, 'gazetter_Final_v3.geojson'), 'name', 'points')
    
    # 하나라도 맵이 로드되지 않으면 종료
    if not all([si_do_map, si_gun_map, points_map]):
        exit()
    
    print("GeoJSON 데이터가 변수에 성공적으로 로드되었습니다.")
except Exception as e:
    print(f"GeoJSON 데이터 로드 중 예기치 않은 오류 발생: {e}")
    exit()

# 3. 위치 이름 정규화를 위한 매핑 딕셔너리
location_mapping = {
    "평양".replace(' ', ''): "평양직할시".replace(' ', ''),
    "평양시".replace(' ', ''): "평양직할시".replace(' ', ''),
    "함흥".replace(' ', ''): "함흥시".replace(' ', ''),
    "해주".replace(' ', ''): "해주시".replace(' ', ''),
    "개성".replace(' ', ''): "개성시".replace(' ', ''),
    "청진".replace(' ', ''): "청진시".replace(' ', ''),
    "서해".replace(' ', ''): "평안남도".replace(' ', ''),
    "백두산".replace(' ', ''): "량강도".replace(' ', ''),
    "금수산궁전".replace(' ', ''): "금수산태양궁전".replace(' ', ''),
    "영변".replace(' ', ''): "녕변".replace(' ', ''),
    "만수대".replace(' ', ''): "만수대의사당".replace(' ', ''),
    "만수대궁전".replace(' ', ''): "만수대의사당".replace(' ', ''),
    "양강도".replace(' ', ''): "량강도".replace(' ', ''),
    "라진".replace(' ', ''): "라진구역".replace(' ', ''),
    "단천".replace(' ', ''): "단천시".replace(' ', ''),
    "나선".replace(' ', ''): "라선특별시".replace(' ', ''),
    "라선".replace(' ', ''): "라선특별시".replace(' ', ''),
    "사리원".replace(' ', ''): "사리원시".replace(' ', ''),
    "원산".replace(' ', ''): "원산시".replace(' ', ''),
    "신의주".replace(' ', ''): "신의주시".replace(' ', '')
}

# 4. 출력 디렉터리 생성 (존재하지 않을 경우)
os.makedirs(output_dir, exist_ok=True)

# 5. 2016년부터 2025년까지 각 파일을 순회하며 작업 수행
for year in range(2016, 2026):
    input_file_path = os.path.join(input_dir, f'new_combined_data_{year}.json')
    output_file_path = os.path.join(output_dir, f'combined_data_{year}_with_coordinates.jsonl')

    if not os.path.exists(input_file_path):
        print(f"경고: {input_file_path} 파일이 존재하지 않아 건너뜁니다.")
        continue

    print(f"\n{year}년 데이터 처리 중...")
    
    try:
        with open(input_file_path, 'r', encoding='utf-8') as infile, \
             open(output_file_path, 'w', encoding='utf-8') as outfile:
            
            year_data = json.load(infile)
            
            for item in year_data:
                unique_locations = list(set(item['locations']))
                locations_with_coords = []
                new_locations_list = []  # 새로운 locations 리스트를 저장할 변수
                
                for loc in unique_locations:
                    coord_info = None
                    normalized_loc = loc.replace(' ', '')
                    
                    if normalized_loc in points_map:
                        coord_info = points_map[normalized_loc]
                    elif location_mapping.get(normalized_loc, normalized_loc) in si_do_map:
                        coord_info = si_do_map[location_mapping.get(normalized_loc, normalized_loc)]
                    elif location_mapping.get(normalized_loc, normalized_loc) in si_gun_map:
                        coord_info = si_gun_map[location_mapping.get(normalized_loc, normalized_loc)]
                    
                    if coord_info:
                        # GeoJSON의 이름으로 locations 리스트와 locations_with_coords 리스트에 추가
                        new_locations_list.append(coord_info['name'])
                        locations_with_coords.append({
                            "name": coord_info['name'], 
                            "coordinates": coord_info['coordinates'],
                            "type": coord_info['type']
                        })
                    else:
                        # 매핑되지 않은 경우, 원본 이름을 그대로 추가
                        new_locations_list.append(loc)
                        locations_with_coords.append({
                            "name": loc,
                            "coordinates": None,
                            "type": None
                        })
                
                # locations 리스트를 새로운 리스트로 덮어쓰기
                item['locations'] = list(set(new_locations_list))
                item['locations_with_coordinates'] = locations_with_coords
                
                outfile.write(json.dumps(item, ensure_ascii=False) + '\n')

        print(f"'{output_file_path}'에 {year}년 데이터 처리가 완료되었습니다. ✅")
    
    except json.JSONDecodeError as e:
        print(f"오류: {year}년 파일 '{input_file_path}'이(가) 유효한 JSON 형식이 아닙니다: {e}")
        continue
    except Exception as e:
        print(f"오류: {year}년 데이터 처리 중 예기치 않은 오류 발생: {e}")
        import traceback
        traceback.print_exc()
        continue

print("\n모든 파일 처리가 완료되었습니다. ✅")

GeoJSON 데이터가 변수에 성공적으로 로드되었습니다.

2016년 데이터 처리 중...
'/home/ds4_sia_nolb/#FINAL_POLARIS/06_Geo_coding/re_combined_data_by_year_mapping_v1_jsonl/combined_data_2016_with_coordinates.jsonl'에 2016년 데이터 처리가 완료되었습니다. ✅

2017년 데이터 처리 중...
'/home/ds4_sia_nolb/#FINAL_POLARIS/06_Geo_coding/re_combined_data_by_year_mapping_v1_jsonl/combined_data_2017_with_coordinates.jsonl'에 2017년 데이터 처리가 완료되었습니다. ✅

2018년 데이터 처리 중...
'/home/ds4_sia_nolb/#FINAL_POLARIS/06_Geo_coding/re_combined_data_by_year_mapping_v1_jsonl/combined_data_2018_with_coordinates.jsonl'에 2018년 데이터 처리가 완료되었습니다. ✅

2019년 데이터 처리 중...
'/home/ds4_sia_nolb/#FINAL_POLARIS/06_Geo_coding/re_combined_data_by_year_mapping_v1_jsonl/combined_data_2019_with_coordinates.jsonl'에 2019년 데이터 처리가 완료되었습니다. ✅

2020년 데이터 처리 중...
'/home/ds4_sia_nolb/#FINAL_POLARIS/06_Geo_coding/re_combined_data_by_year_mapping_v1_jsonl/combined_data_2020_with_coordinates.jsonl'에 2020년 데이터 처리가 완료되었습니다. ✅

2021년 데이터 처리 중...
'/home/ds4_sia_nolb/#FINAL_POLARIS/06_Geo_coding/re_co

In [2]:
import json
import os
import geopandas as gpd
from shapely.geometry import Point
from tqdm import tqdm

# --- 1. 파일 경로 설정 ---
BASE_DIR = '/home/ds4_sia_nolb/#FINAL_POLARIS/06_Geo_coding'
GEOJSON_DIR = os.path.join(BASE_DIR, 'Geosjon_data')
# 입력 디렉토리는 JSONL 파일
DATA_INPUT_DIR = os.path.join(BASE_DIR, 're_combined_data_by_year_mapping_v1_jsonl')
# 출력 디렉토리는 JSONL 파일
DATA_OUTPUT_DIR = os.path.join(BASE_DIR, 're_combined_data_by_year_mapping_v2_jsonl')

SI_DO_PATH = os.path.join(GEOJSON_DIR, 'dprk_si_do.geojson')
SI_GUN_PATH = os.path.join(GEOJSON_DIR, 'dprk_si_gun.geojson')

# 출력 디렉터리가 없으면 생성
if not os.path.exists(DATA_OUTPUT_DIR):
    os.makedirs(DATA_OUTPUT_DIR)

# --- 2. GeoJSON 데이터 불러오기 ---
print("GeoJSON 파일 불러오는 중...")
try:
    si_do_gdf = gpd.read_file(SI_DO_PATH)
    si_do_gdf = si_do_gdf[['NL_NAME_1', 'geometry']].rename(columns={'NL_NAME_1': 'name'})
    si_gun_gdf = gpd.read_file(SI_GUN_PATH)
    si_gun_gdf['full_name'] = si_gun_gdf['NL_NAME_1'] + ' ' + si_gun_gdf['NL_NAME_2']
    si_gun_gdf = si_gun_gdf[['NL_NAME_2', 'full_name', 'geometry']].rename(columns={'NL_NAME_2': 'name'})
    print("GeoJSON 파일 불러오기 완료.")
except Exception as e:
    print(f"GeoJSON 파일을 불러오는 중 오류가 발생했습니다: {e}")
    exit()

# --- 3. 데이터 디렉토리 순회 ---
data_files = [f for f in os.listdir(DATA_INPUT_DIR) if f.endswith('.jsonl')]
print(f"총 {len(data_files)}개의 데이터 파일을 처리합니다.")

for file_name in data_files:
    input_file_path = os.path.join(DATA_INPUT_DIR, file_name)
    output_file_path = os.path.join(DATA_OUTPUT_DIR, file_name)
    print(f"\n--- {file_name} 파일 처리 시작 ---")

    try:
        with open(input_file_path, 'r', encoding='utf-8') as infile, \
             open(output_file_path, 'w', encoding='utf-8') as outfile:
            
            added_locations_total = set()
            lines = infile.readlines()
            
            for line in tqdm(lines, desc=f"{file_name} 처리 중"):
                if not line.strip(): continue
                item = json.loads(line)
                
                locations_with_coords = item.get('locations_with_coordinates', [])
                locations_names = {loc['name'] for loc in locations_with_coords if loc['name']}
                
                points_to_check = [loc for loc in locations_with_coords if loc.get('type') == 'points' and loc.get('coordinates')]
                added_locations_current = set()

                for point_loc in points_to_check:
                    try:
                        lon, lat = point_loc['coordinates']
                        point_geom = Point(lon, lat)

                        si_gun_matches = si_gun_gdf[si_gun_gdf.contains(point_geom)]
                        for _, row in si_gun_matches.iterrows():
                            si_gun_name_to_save = row['name']
                            si_gun_full_name = row['full_name']
                            if si_gun_name_to_save not in locations_names:
                                item['locations'].append(si_gun_name_to_save)
                                item['locations_with_coordinates'].append({
                                    'name': si_gun_name_to_save,
                                    'coordinates': [list(p.exterior.coords) for p in row['geometry'].geoms] if row['geometry'].type == 'MultiPolygon' else list(row['geometry'].exterior.coords),
                                    'type': 'si_gun'
                                })
                                locations_names.add(si_gun_name_to_save)
                                added_locations_current.add(si_gun_full_name)

                        si_do_matches = si_do_gdf[si_do_gdf.contains(point_geom)]
                        for _, row in si_do_matches.iterrows():
                            si_do_name = row['name']
                            if si_do_name not in locations_names:
                                item['locations'].append(si_do_name)
                                item['locations_with_coordinates'].append({
                                    'name': si_do_name,
                                    'coordinates': [list(p.exterior.coords) for p in row['geometry'].geoms] if row['geometry'].type == 'MultiPolygon' else list(row['geometry'].exterior.coords),
                                    'type': 'si_do'
                                })
                                locations_names.add(si_do_name)
                                added_locations_current.add(si_do_name)
                    except Exception as e:
                        print(f"경고: {point_loc.get('name')} 좌표 처리 중 오류 발생: {e}")
                        continue
                
                added_locations_total.update(added_locations_current)
                outfile.write(json.dumps(item, ensure_ascii=False) + '\n')

    except Exception as e:
        print(f"오류: {file_name} 파일 처리 중 예기치 않은 오류 발생: {e}")
        import traceback
        traceback.print_exc()
        continue

    print(f"\n--- {output_file_path} 파일 저장 완료 ---")
    if added_locations_total:
        print(f"✅ {file_name}에서 다음 지역들이 추가되었습니다:")
        for loc in sorted(list(added_locations_total)):
            print(f"   - {loc}")
    else:
        print(f"❌ {file_name}에 추가된 지역이 없습니다.")

print("\n모든 파일 처리가 완료되었습니다.")

GeoJSON 파일 불러오는 중...
GeoJSON 파일 불러오기 완료.
총 10개의 데이터 파일을 처리합니다.

--- combined_data_2016_with_coordinates.jsonl 파일 처리 시작 ---


combined_data_2016_with_coordinates.jsonl 처리 중:   0%|          | 0/7385 [00:00<?, ?it/s]/tmp/ipykernel_7983/4212083423.py:74: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  'coordinates': [list(p.exterior.coords) for p in row['geometry'].geoms] if row['geometry'].type == 'MultiPolygon' else list(row['geometry'].exterior.coords),
/tmp/ipykernel_7983/4212083423.py:74: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  'coordinates': [list(p.exterior.coords) for p in row['geometry'].geoms] if row['geometry'].type == 'MultiPolygon' else list(row['geometry'].exterior.coords),
/tmp/ipykernel_7983/4212083423.py:74: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  'coordinates': [list(p.exterior.coords) f


--- /home/ds4_sia_nolb/#FINAL_POLARIS/06_Geo_coding/re_combined_data_by_year_mapping_v2_jsonl/combined_data_2016_with_coordinates.jsonl 파일 저장 완료 ---
✅ combined_data_2016_with_coordinates.jsonl에서 다음 지역들이 추가되었습니다:
   - 강원도
   - 개성공업지구
   - 개성시 판문군
   - 평안북도
   - 평안북도 구성시
   - 평안북도 녕변군
   - 평안북도 철산군
   - 평양직할시
   - 평양직할시 주변구역
   - 함경남도
   - 함경남도 신포시
   - 함경북도
   - 함경북도 무산군
   - 함경북도 어랑군

--- combined_data_2019_with_coordinates.jsonl 파일 처리 시작 ---


combined_data_2019_with_coordinates.jsonl 처리 중:   0%|          | 0/5786 [00:00<?, ?it/s]/tmp/ipykernel_7983/4212083423.py:74: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  'coordinates': [list(p.exterior.coords) for p in row['geometry'].geoms] if row['geometry'].type == 'MultiPolygon' else list(row['geometry'].exterior.coords),
/tmp/ipykernel_7983/4212083423.py:74: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  'coordinates': [list(p.exterior.coords) for p in row['geometry'].geoms] if row['geometry'].type == 'MultiPolygon' else list(row['geometry'].exterior.coords),
/tmp/ipykernel_7983/4212083423.py:74: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  'coordinates': [list(p.exterior.coords) f


--- /home/ds4_sia_nolb/#FINAL_POLARIS/06_Geo_coding/re_combined_data_by_year_mapping_v2_jsonl/combined_data_2019_with_coordinates.jsonl 파일 저장 완료 ---
✅ combined_data_2019_with_coordinates.jsonl에서 다음 지역들이 추가되었습니다:
   - 강원도
   - 강원도 원산시
   - 개성공업지구
   - 개성시 판문군
   - 자강도
   - 자강도 향산군
   - 자강도 희천시
   - 평안남도
   - 평안남도 순천시
   - 평안북도
   - 평안북도 녕변군
   - 평안북도 운전군
   - 평안북도 철산군
   - 평양직할시
   - 평양직할시 주변구역
   - 함경남도
   - 함경남도 신포시
   - 함경남도 정평군
   - 함경남도 함흥시
   - 함경북도
   - 함경북도 어랑군
   - 황해북도
   - 황해북도 봉산군

--- combined_data_2017_with_coordinates.jsonl 파일 처리 시작 ---


...


--- /home/ds4_sia_nolb/#FINAL_POLARIS/06_Geo_coding/re_combined_data_by_year_mapping_v2_jsonl/combined_data_2024_with_coordinates.jsonl 파일 저장 완료 ---
✅ combined_data_2024_with_coordinates.jsonl에서 다음 지역들이 추가되었습니다:
   - 강원도 판교군
   - 개성공업지구
   - 개성시 판문군
   - 남포시 온천군
   - 자강도 룡림군
   - 평안남도
   - 평안남도 순천시
   - 평안남도 천리마구역
   - 평안북도
   - 평안북도 녕변군
   - 평안북도 의주군
   - 평안북도 철산군
   - 평양직할시
   - 평양직할시 주변구역
   - 함경남도
   - 함경남도 신포시
   - 함경남도 정평군
   - 함경북도
   - 함경북도 어랑군

--- combined_data_2025_with_coordinates.jsonl 파일 처리 시작 ---


combined_data_2025_with_coordinates.jsonl 처리 중:   0%|          | 0/1016 [00:00<?, ?it/s]/tmp/ipykernel_7983/4212083423.py:74: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  'coordinates': [list(p.exterior.coords) for p in row['geometry'].geoms] if row['geometry'].type == 'MultiPolygon' else list(row['geometry'].exterior.coords),
/tmp/ipykernel_7983/4212083423.py:74: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  'coordinates': [list(p.exterior.coords) for p in row['geometry'].geoms] if row['geometry'].type == 'MultiPolygon' else list(row['geometry'].exterior.coords),
/tmp/ipykernel_7983/4212083423.py:74: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  'coordinates': [list(p.exterior.coords) f


--- /home/ds4_sia_nolb/#FINAL_POLARIS/06_Geo_coding/re_combined_data_by_year_mapping_v2_jsonl/combined_data_2025_with_coordinates.jsonl 파일 저장 완료 ---
✅ combined_data_2025_with_coordinates.jsonl에서 다음 지역들이 추가되었습니다:
   - 강원도
   - 개성공업지구
   - 개성시 판문군
   - 평안남도
   - 평안남도 천리마구역
   - 평안북도
   - 평안북도 의주군
   - 평안북도 철산군
   - 평양직할시
   - 평양직할시 주변구역
   - 함경남도 정평군
   - 함경북도
   - 함경북도 어랑군

--- combined_data_2022_with_coordinates.jsonl 파일 처리 시작 ---


combined_data_2022_with_coordinates.jsonl 처리 중:   0%|          | 0/4391 [00:00<?, ?it/s]/tmp/ipykernel_7983/4212083423.py:74: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  'coordinates': [list(p.exterior.coords) for p in row['geometry'].geoms] if row['geometry'].type == 'MultiPolygon' else list(row['geometry'].exterior.coords),
/tmp/ipykernel_7983/4212083423.py:74: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  'coordinates': [list(p.exterior.coords) for p in row['geometry'].geoms] if row['geometry'].type == 'MultiPolygon' else list(row['geometry'].exterior.coords),
/tmp/ipykernel_7983/4212083423.py:74: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  'coordinates': [list(p.exterior.coords) f


--- /home/ds4_sia_nolb/#FINAL_POLARIS/06_Geo_coding/re_combined_data_by_year_mapping_v2_jsonl/combined_data_2022_with_coordinates.jsonl 파일 저장 완료 ---
✅ combined_data_2022_with_coordinates.jsonl에서 다음 지역들이 추가되었습니다:
   - 개성공업지구
   - 개성시 판문군
   - 남포시 온천군
   - 자강도 시중군
   - 평안남도
   - 평안남도 순천시
   - 평안북도
   - 평안북도 녕변군
   - 평안북도 의주군
   - 평안북도 철산군
   - 평양직할시
   - 평양직할시 주변구역
   - 함경남도
   - 함경남도 신포시
   - 함경남도 함흥시
   - 함경북도
   - 함경북도 어랑군

모든 파일 처리가 완료되었습니다.


In [5]:
import json
import os
import glob

def extract_article_data(file_path):
    """
    주어진 JSONL 파일에서 각 기사의 특정 필드('id_', 'title', 'url', 'pubDate', 'locations')를 추출합니다.

    Args:
        file_path (str): JSONL 파일의 경로.

    Returns:
        list: 추출된 필드들을 담은 딕셔너리들의 리스트.
              오류 발생 시 빈 리스트를 반환합니다.
    """
    extracted_data = []
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f:
                # 각 줄이 하나의 JSON 객체이므로 json.loads()를 사용하여 파싱
                item = json.loads(line.strip())
                # 필요한 필드가 모두 존재하는지 확인
                if all(key in item for key in ['id_', 'title', 'url', 'pubDate', 'locations']):
                    extracted_data.append({
                        'id_': item['id_'],
                        'title': item['title'],
                        'url': item['url'],
                        'pubDate': item['pubDate'],
                        'locations': item['locations']
                    })
    except FileNotFoundError:
        print(f"오류: 파일을 찾을 수 없습니다 - {file_path}")
    except json.JSONDecodeError as e:
        print(f"오류: JSON 디코딩에 실패했습니다. 파일 형식을 확인해주세요. (파일: {file_path}, 오류: {e})")
    except Exception as e:
        print(f"예상치 못한 오류가 발생했습니다: {e}")
        
    return extracted_data

def save_results(data, output_file_path):
    """
    추출된 데이터를 JSONL 형식으로 파일에 저장합니다.
    각 JSON 객체가 한 줄에 하나씩 작성됩니다.

    Args:
        data (list): 저장할 딕셔너리 리스트.
        output_file_path (str): 결과를 저장할 파일의 경로.
    """
    try:
        # 출력 디렉토리가 존재하지 않으면 생성합니다.
        output_dir = os.path.dirname(output_file_path)
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
            print(f"디렉토리 '{output_dir}'를 생성했습니다.")

        with open(output_file_path, 'w', encoding='utf-8') as f:
            for item in data:
                f.write(json.dumps(item, ensure_ascii=False) + '\n')
        print(f"결과가 성공적으로 {output_file_path}에 저장되었습니다.")
        print(f"총 {len(data)}개의 기사 데이터가 저장되었습니다.")
    except Exception as e:
        print(f"오류: 파일 저장 중 문제가 발생했습니다: {e}")

if __name__ == "__main__":
    # 입력 디렉토리 경로
    input_directory = '/home/ds4_sia_nolb/#FINAL_POLARIS/06_Geo_coding/re_combined_data_by_year_mapping_v2_jsonl'
    # 출력 디렉토리 경로
    output_directory = '/home/ds4_sia_nolb/#FINAL_POLARIS/06_Geo_coding/re_combined_data_by_year_mapping_v3_jsonl'
    
    # 입력 디렉토리 내의 모든 .jsonl 파일을 찾습니다.
    file_list = glob.glob(os.path.join(input_directory, '*.jsonl'))
    
    if not file_list:
        print(f"오류: '{input_directory}' 디렉토리에서 .jsonl 파일을 찾을 수 없습니다.")
    else:
        print(f"'{input_directory}' 디렉토리에서 {len(file_list)}개의 파일을 찾았습니다.")
        
        # 출력 디렉토리가 존재하지 않으면 생성합니다.
        if not os.path.exists(output_directory):
            os.makedirs(output_directory)
            print(f"디렉토리 '{output_directory}'를 생성했습니다.")

        for i, file_path in enumerate(file_list):
            base_filename = os.path.basename(file_path)
            # 입력 파일 이름에 '_extracted.jsonl' 접미사를 붙여 출력 파일 경로를 만듭니다.
            output_filename = f"{os.path.splitext(base_filename)[0]}_extracted.jsonl"
            output_file_path = os.path.join(output_directory, output_filename)
            
            print(f"\n[{i+1}/{len(file_list)}] '{base_filename}' 파일 처리 중...")
            
            extracted_data = extract_article_data(file_path)
            
            if extracted_data:
                save_results(extracted_data, output_file_path)
            else:
                print(f"경고: '{base_filename}'에서 추출할 데이터가 없거나 오류가 발생했습니다.")

'/home/ds4_sia_nolb/#FINAL_POLARIS/06_Geo_coding/re_combined_data_by_year_mapping_v2_jsonl' 디렉토리에서 10개의 파일을 찾았습니다.

[1/10] 'combined_data_2016_with_coordinates.jsonl' 파일 처리 중...
결과가 성공적으로 /home/ds4_sia_nolb/#FINAL_POLARIS/06_Geo_coding/re_combined_data_by_year_mapping_v3_jsonl/combined_data_2016_with_coordinates_extracted.jsonl에 저장되었습니다.
총 7385개의 기사 데이터가 저장되었습니다.

[2/10] 'combined_data_2019_with_coordinates.jsonl' 파일 처리 중...
결과가 성공적으로 /home/ds4_sia_nolb/#FINAL_POLARIS/06_Geo_coding/re_combined_data_by_year_mapping_v3_jsonl/combined_data_2019_with_coordinates_extracted.jsonl에 저장되었습니다.
총 5786개의 기사 데이터가 저장되었습니다.

[3/10] 'combined_data_2017_with_coordinates.jsonl' 파일 처리 중...
결과가 성공적으로 /home/ds4_sia_nolb/#FINAL_POLARIS/06_Geo_coding/re_combined_data_by_year_mapping_v3_jsonl/combined_data_2017_with_coordinates_extracted.jsonl에 저장되었습니다.
총 7924개의 기사 데이터가 저장되었습니다.

[4/10] 'combined_data_2021_with_coordinates.jsonl' 파일 처리 중...
결과가 성공적으로 /home/ds4_sia_nolb/#FINAL_POLARIS/06_Geo_coding/re_combined

In [6]:
import json
import os
import glob

# --- 1. 파일 경로 설정 ---
# 변환할 JSONL 파일들이 있는 디렉토리
input_dir = '/home/ds4_sia_nolb/#FINAL_POLARIS/06_Geo_coding/re_combined_data_by_year_mapping_v3_jsonl'
# 변환된 JSON 파일을 저장할 디렉토리
output_dir = '/home/ds4_sia_nolb/#FINAL_POLARIS/06_Geo_coding/re_combined_data_by_year_mapping_v4_json'

# 출력 디렉터리가 없으면 생성
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# --- 2. 모든 .jsonl 파일 찾기 ---
# glob 모듈을 사용하여 input_dir 내의 모든 .jsonl 파일 경로를 가져옵니다.
jsonl_files = glob.glob(os.path.join(input_dir, '*.jsonl'))

print(f"총 {len(jsonl_files)}개의 .jsonl 파일을 찾았습니다.")

# --- 3. 각 파일을 순회하며 변환 ---
for jsonl_file_path in jsonl_files:
    # 파일 이름과 확장자를 분리하여 새로운 .json 파일 경로를 만듭니다.
    base_name = os.path.basename(jsonl_file_path)
    file_name_without_ext = os.path.splitext(base_name)[0]
    output_file_path = os.path.join(output_dir, f"{file_name_without_ext}.json")

    print(f"\n--- '{base_name}' 파일 변환 시작 ---")

    try:
        # JSONL 파일의 내용을 한 줄씩 읽어 리스트에 저장합니다.
        data_list = []
        with open(jsonl_file_path, 'r', encoding='utf-8') as infile:
            for line in infile:
                # 빈 줄이거나 유효하지 않은 JSON이 있으면 건너뜁니다.
                if line.strip():
                    data_list.append(json.loads(line))
        
        # 리스트를 하나의 JSON 배열로 파일에 저장합니다.
        with open(output_file_path, 'w', encoding='utf-8') as outfile:
            json.dump(data_list, outfile, indent=4, ensure_ascii=False)
            
        print(f"✅ '{output_file_path}' 파일로 변환 완료.")

    except json.JSONDecodeError as e:
        print(f"❌ 오류: '{base_name}' 파일이 유효한 JSONL 형식이 아닙니다: {e}")
        continue
    except Exception as e:
        print(f"❌ 오류: '{base_name}' 파일 변환 중 예기치 않은 오류 발생: {e}")
        continue

print("\n모든 파일 변환이 완료되었습니다.")

총 10개의 .jsonl 파일을 찾았습니다.

--- 'combined_data_2016_with_coordinates_extracted.jsonl' 파일 변환 시작 ---
✅ '/home/ds4_sia_nolb/#FINAL_POLARIS/06_Geo_coding/re_combined_data_by_year_mapping_v4_json/combined_data_2016_with_coordinates_extracted.json' 파일로 변환 완료.

--- 'combined_data_2021_with_coordinates_extracted.jsonl' 파일 변환 시작 ---
✅ '/home/ds4_sia_nolb/#FINAL_POLARIS/06_Geo_coding/re_combined_data_by_year_mapping_v4_json/combined_data_2021_with_coordinates_extracted.json' 파일로 변환 완료.

--- 'combined_data_2018_with_coordinates_extracted.jsonl' 파일 변환 시작 ---
✅ '/home/ds4_sia_nolb/#FINAL_POLARIS/06_Geo_coding/re_combined_data_by_year_mapping_v4_json/combined_data_2018_with_coordinates_extracted.json' 파일로 변환 완료.

--- 'combined_data_2017_with_coordinates_extracted.jsonl' 파일 변환 시작 ---
✅ '/home/ds4_sia_nolb/#FINAL_POLARIS/06_Geo_coding/re_combined_data_by_year_mapping_v4_json/combined_data_2017_with_coordinates_extracted.json' 파일로 변환 완료.

--- 'combined_data_2023_with_coordinates_extracted.jsonl' 파일 변환 시작